In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
from selenium.webdriver.chrome.options import Options
from tempfile import mkdtemp

# Configura el WebDriver (asegúrate de que la ruta a chromedriver sea correcta)
chromedriver_autoinstaller.install()

# Abre la página de Twitter
options = webdriver.ChromeOptions()

options.add_argument("--no-sandbox")
options.add_argument("--headless=new")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--log-level=3")
options.add_argument("--disable-popup-blocking")
options.add_argument("--disable-notifications")
options.add_argument("--disable-extensions")
options.add_argument("--disable-background-networking")
options.add_argument("--ignore-certificate-errors")
options.add_argument(f"--user-data-dir={mkdtemp()}")
options.add_argument(f"--data-path={mkdtemp()}")
options.add_argument(f"--disk-cache-dir={mkdtemp()}")
options.add_argument("--remote-debugging-port=9226")

driver = webdriver.Chrome(
    options=options,
)

driver.get("https://x.com/antodiazcano")

# Espera a que la página cargue (puedes ajustar el tiempo según sea necesario)
time.sleep(5)

# Encuentra los tweets (inspecciona el HTML para obtener la clase correcta)
tweets = driver.find_elements(By.XPATH, '//div[@data-testid="tweet"]')

# Imprime el texto de los tweets
for tweet in tweets:
    print(tweet.text)

# Cierra el navegador
driver.quit()

ValueError: No chrome executable found on PATH

In [13]:
import requests
from bs4 import BeautifulSoup

# URL de tu perfil de Twitter
url = "https://x.com/tunguz"

# Hacer la solicitud
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Encontrar los tweets usando la clase correcta
tweets = soup.find_all("div", {"class": "css-1dbjc4n r-1iusvr4 r-16y2uox"})

for tweet in tweets:
    text = tweet.find("div", {"lang": "es"})  # Ajusta según el idioma y la estructura
    if text:
        print(text.text)